In [1]:
from results.scalar_reg_results import ResultGenerator
from data.turtle_data_loading import get_file_paths, TurtleDataset
from metrics.metrics import compare

import os
import torch
# from pytorch_msssim import ssim # This cpu SSIM is a bit faster than skimage SSIM while still producing consistent results.
# from skimage.metrics import peak_signal_noise_ratio # skimage PSNR is surprisingly fast, only a bit slower than ignite cuda PSNR
# from skimage.metrics import structural_similarity # assumed as standard 
import numpy as np
from pprint import pprint


import warnings
warnings.filterwarnings("ignore")

# import sys

# # Function to suppress stdout and stderr
# class SuppressOutput:
#     def __enter__(self):
#         self._original_stdout = sys.stdout
#         self._original_stderr = sys.stderr
#         self._original_stdout_fd = sys.stdout.fileno()
#         self._original_stderr_fd = sys.stderr.fileno()

#         self._null_fd = os.open(os.devnull, os.O_RDWR)
#         os.dup2(self._null_fd, self._original_stdout_fd)
#         os.dup2(self._null_fd, self._original_stderr_fd)

#     def __exit__(self, exc_type, exc_val, exc_tb):
#         os.dup2(self._original_stdout_fd, self._original_stdout_fd)
#         os.dup2(self._original_stderr_fd, self._original_stderr_fd)
#         os.close(self._null_fd)
#         sys.stdout = self._original_stdout
#         sys.stderr = self._original_stderr

Using new version 2 of turtle_data_generate.py


In [2]:
from turtle_id_test_config import upper_level, test_scalar_reg_path

data_path=f"{upper_level}/../data/turtle_id_2022/turtles-data/data"

# result_path = "../test"
result_path = test_scalar_reg_path
os.makedirs(result_path, exist_ok=True)

# # "T": 128, "train_num_samples": 500, "val_num_samples": 50, "resize_square": 256, "sigmas": "[0.05, 0.1, 0.15, 0.2, 0.25, 0.3]", "init_filters": 32, "n_blocks": 3,
# model_path=f"{upper_level}/tmp_2/model-turtle_id_2022-2024_06_21_14_31_50/model_epoch_30.pt"


# https://wandb.ai/wof/turtle_id_2022/runs/cfbkuz9y
model_date, epoch = "06_24_19_56_05", 50


model_path=f"{upper_level}/models/model-turtle_id_2022-2024_{model_date}/model_epoch_{epoch}.pt"


# size = 256
size = 512
num_samples = 100 # 0 for all

sigmas = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

# default_device = "cuda"
default_device = "cpu"
torch.set_default_device(default_device)

num_threads = 4

In [3]:
!pwd

/mnt/h/GIT/DISSERTATION/LearningRegularizationParameterMaps/turtle_id/test


In [4]:
file_paths:dict = get_file_paths(data_path, "test", num_samples, sigmas, size)
# file_paths:dict = {
#     0: [f'images_crop_resize_{size}_greyscale/t008/PaXakrkxqN.JPG'],
#     0.05: [f'images_crop_resize_{size}_greyscale_noisy_0_05/t008/PaXakrkxqN.JPG'],
#     0.1: [f'images_crop_resize_{size}_greyscale_noisy_0_1/t008/PaXakrkxqN.JPG'],
#     0.15: [f'images_crop_resize_{size}_greyscale_noisy_0_15/t008/PaXakrkxqN.JPG'],
#     0.2: [f'images_crop_resize_{size}_greyscale_noisy_0_2/t008/PaXakrkxqN.JPG'],
#     0.25: [f'images_crop_resize_{size}_greyscale_noisy_0_25/t008/PaXakrkxqN.JPG'],
#     0.3: [f'images_crop_resize_{size}_greyscale_noisy_0_3/t008/PaXakrkxqN.JPG'],
# }

Loading original image paths in images_crop_resize_512_greyscale 

100%|██████████| 100/100 [00:00<00:00, 1095118.54it/s]


Loading noisy image paths sigma=0.05 in images_crop_resize_512_greyscale_noisy_0_05 

100%|██████████| 100/100 [00:00<00:00, 1375181.64it/s]


Loading noisy image paths sigma=0.1 in images_crop_resize_512_greyscale_noisy_0_1 

...

100%|██████████| 100/100 [00:00<00:00, 1487341.84it/s]


Loading noisy image paths sigma=0.25 in images_crop_resize_512_greyscale_noisy_0_25 

100%|██████████| 100/100 [00:00<00:00, 1412223.57it/s]


Loading noisy image paths sigma=0.3 in images_crop_resize_512_greyscale_noisy_0_3 

100%|██████████| 100/100 [00:00<00:00, 1348650.80it/s]


In [5]:
# pprint(file_paths)

In [6]:
test_dataset = TurtleDataset(data_path, file_paths, device=default_device)

Loading original images 

100%|██████████| 100/100 [00:00<00:00, 178.88it/s]


Loading noisy images sigma=0.05 

100%|██████████| 100/100 [00:00<00:00, 193.68it/s]


Loading noisy images sigma=0.1 

...

100%|██████████| 100/100 [00:00<00:00, 183.05it/s]


Loading noisy images sigma=0.25 

100%|██████████| 100/100 [00:00<00:00, 172.13it/s]


Loading noisy images sigma=0.3 

100%|██████████| 100/100 [00:00<00:00, 171.19it/s]


In [7]:
result_generator = ResultGenerator(
    min_lambda=0, max_lambda=0.4, num_lambdas=81,
    cmp_func=compare, saving_denoised=True,
    in_path=data_path, out_path=result_path,
    file_paths=file_paths, returning_denoised_PILs=False,
)

100%|██████████| 100/100 [00:01<00:00, 90.91it/s]


In [8]:
# with SuppressOutput():
#     results = result_generator.process_samples(num_threads=num_threads)
results = result_generator.process_samples(num_threads=num_threads)

2024-06-27 02:13:43,418 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,422 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,527 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,528 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,575 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,576 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,613 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,615 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,665 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,667 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,714 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,715 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,754 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,756 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,799 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,800 - DEBUG - STREAM b'IDAT' 41 65536
2024-06-27 02:13:43,860 - DEBUG - STREAM b'IHDR' 16 13
2024-06-27 02:13:43,861 - DEBUG - STREAM 

In [9]:
ar = [1, 2, 3]
ar_np = np.array(ar)
ar_np_2 = np.array(ar_np)
print(ar_np)
print(ar_np_2)

[1 2 3]
[1 2 3]


In [10]:
import pandas as pd
test_df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": [4, 5, 6],
    "c": [7, 8, 9]
})
# Add a new column
test_df["d"] = [10, 11, 12]
print(test_df)

test_df_2 = pd.DataFrame({
    "d": [10, 11, 12],
    "a": [1, 2, 3],
    "b": [4, 5, 6],
    "c": [7, 8, 9]
})

test_df_3 = pd.concat([test_df, test_df_2], ignore_index=True)

print(test_df_3)

   a  b  c   d
0  1  4  7  10
1  2  5  8  11
2  3  6  9  12
   a  b  c   d
0  1  4  7  10
1  2  5  8  11
2  3  6  9  12
3  1  4  7  10
4  2  5  8  11
5  3  6  9  12
